<a href="https://colab.research.google.com/github/abdipourasl/Alzheimer/blob/main/MNE_Preprocessing_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-success">
<h1>EEG Preprocessing with MNE</h1>
</div>

In [316]:
! pip install mne

In [317]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [318]:
%matplotlib inline
import os
import os.path as op
import mne
import numpy as np
import pandas as pd

### 1. Importing Raw  Data

>You need to provide your own EEG data with BrainVision format which have three files named  .vhdr, .vmrk and .eeg

<div class="alert alert-block alert-info">
<b>Tip:</b> You can change read_raw_brainvision() <a href="https://mne.tools/stable/auto_tutorials/io/plot_20_reading_eeg_data.html" title="Data EEG formats"> module based on your data format</a>
</div>

In [319]:
import zipfile

# Replace 'your_zip_file.zip' with the path to your zip file
zip_path = '/content/drive/MyDrive/Alzheimer 2/Database_AD_MCI_HC_Wake_EDFraw_complete-20240930T174905Z-002.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')  # Specify your destination folder

In [320]:
examples_dir = '/content/Database_AD_MCI_HC_Wake_EDFraw_complete/DatabaseCorretto/IDpz_102'  # Path to the raw EEG Data folder
edf_file = op.join(examples_dir, 'IDpz102_W2OA.edf')  # Path to the raw EEG EDF file

# Load the EDF file instead of BrainVision file
raw = mne.io.read_raw_edf(edf_file, preload=True)  # preload=True loads the data into memory


Extracting EDF parameters from /content/Database_AD_MCI_HC_Wake_EDFraw_complete/DatabaseCorretto/IDpz_102/IDpz102_W2OC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 88831  =      0.000 ...   346.996 secs...


<ipython-input-320-bfdd950b41e8>:5: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)  # preload=True loads the data into memory
<ipython-input-320-bfdd950b41e8>:5: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=True)  # preload=True loads the data into memory
<ipython-input-320-bfdd950b41e8>:5: RuntimeWarning: Highpass cutoff frequency 40.0 is greater than lowpass cutoff frequency 1.0, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(edf_file, preload=True)  # preload=True loads the data into memory


In [321]:
# Check channel names to confirm correct labeling
print(raw.info['ch_names'])


['Fp1-A1+A2', 'Fp2-A1+A2', 'F3-A1+A2', 'F4-A1+A2', 'Fz-A1+A2', 'F7-A1+A2', 'F8-A1+A2', 'C3-A1+A2', 'C4-A1+A2', 'Cz-A1+A2', 'P3-A1+A2', 'P4-A1+A2', 'Pz-A1+A2', 'T3-A1+A2', 'T4-A1+A2', 'T5-A1+A2', 'T6-A1+A2', 'O1-A1+A2', 'O2-A1+A2', 'A1-A1+A2', 'A2-A1+A2', 'EMG1+', 'EOG1+', 'ECG1+', 'SpO2+']


In [322]:
# Setting multiple channel types at once
# raw.set_channel_types({'ECG1+': 'ecg', 'SpO2+': 'bio'})

###### Consider just first 19 channels in preprocessing

In [323]:
# Step 1: Remove unwanted channels
# Specify the channels to drop (SpO2+, EOG1+, EMG1+)
channels_to_drop = ['SpO2+', 'EOG1+', 'EMG1+', 'ECG1+']
raw.drop_channels(channels_to_drop)

# # Step 2: Re-reference using the first 19 EEG channels (exclude ECG)
# # First, get the names of the first 19 channels (all channels except ECG and reference ones)
# ref_channels = [
#     'Fp1-A1+A2', 'Fp2-A1+A2', 'F3-A1+A2', 'F4-A1+A2', 'Fz-A1+A2', 'F7-A1+A2',
#     'F8-A1+A2', 'C3-A1+A2', 'C4-A1+A2', 'Cz-A1+A2', 'P3-A1+A2', 'P4-A1+A2',
#     'Pz-A1+A2', 'T3-A1+A2', 'T4-A1+A2', 'T5-A1+A2', 'T6-A1+A2', 'O1-A1+A2', 'O2-A1+A2'
# ]

<RawEDF | IDpz102_W2OC.edf, 21 x 88832 (347.0 s), ~14.3 MB, data loaded>

### 4. Rereference

In [324]:
# raw.set_eeg_reference('average', projection=True).apply_proj()  # re-referencing with the virtual average reference
raw.set_eeg_reference(ref_channels=['A1-A1+A2', 'A2-A1+A2'])
# raw.set_eeg_reference(ref_channels=['A1-G2', 'A2-G2'])  # Specify the reference channels
raw.drop_channels(['A1-A1+A2', 'A2-A1+A2'])

EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


<RawEDF | IDpz102_W2OC.edf, 19 x 88832 (347.0 s), ~12.9 MB, data loaded>

In [ ]:
!pip install pyprep
from pyprep.prep_pipeline import PrepPipeline
import mne


# Print the current channel names
print("Current channel names:", raw.ch_names)
# Define a mapping from your channel names to the standard montage
channel_mapping = {
    'Fp1-A1+A2': 'Fp1',
    'Fp2-A1+A2': 'Fp2',
    'F3-A1+A2': 'F3',
    'F4-A1+A2': 'F4',
    'Fz-A1+A2': 'Fz',
    'F7-A1+A2': 'F7',
    'F8-A1+A2': 'F8',
    'C3-A1+A2': 'C3',
    'C4-A1+A2': 'C4',
    'Cz-A1+A2': 'Cz',
    'P3-A1+A2': 'P3',
    'P4-A1+A2': 'P4',
    'Pz-A1+A2': 'Pz',
    'T3-A1+A2': 'T3',
    'T4-A1+A2': 'T4',
    'T5-A1+A2': 'T5',
    'T6-A1+A2': 'T6',
    'O1-A1+A2': 'O1',
    'O2-A1+A2': 'O2',
    # Include any other mappings if necessary
}

# Rename the channels in the raw object
raw.rename_channels(channel_mapping)

# Load a standard montage (e.g., the 10-20 system)
montage = mne.channels.make_standard_montage('standard_1020')

# Set the montage to the raw object
raw.set_montage(montage)

# Define preprocessing parameters for the pipeline
prep_params = {
    'do_detrend': True,
    'detrend_cutoff': 0.5,
    'asr': {
        'threshold': 3.0,  # ASR threshold for detecting bad channels
    },
    'ref_chs': raw.ch_names,  # Set reference channels to all available channels
    'reref_chs': [],  # Specify channels to re-reference if needed
    'line_freqs': [50]  # Specify the line frequency for notch filtering (50 Hz)
}

# Initialize the PREP pipeline
prep = PrepPipeline(raw, montage=montage, prep_params=prep_params)

# Fit the pipeline
try:
    prep.fit()  # This line may raise errors if there are issues
except Exception as e:
    print("Error during fitting:", e)

# Check for bad channels directly from the raw object
bad_channels = raw.info['bads']
print("Bad channels detected by ASR:", bad_channels)

# If you want to mark them in the raw object
raw.info['bads'] = bad_channels

# Interpolate bad channels using data from surrounding electrodes
# raw.interpolate_bads()

# Optionally save the cleaned raw object
# raw.save('cleaned_eeg.fif', overwrite=True)


Current channel names: ['Fp1-A1+A2', 'Fp2-A1+A2', 'F3-A1+A2', 'F4-A1+A2', 'Fz-A1+A2', 'F7-A1+A2', 'F8-A1+A2', 'C3-A1+A2', 'C4-A1+A2', 'Cz-A1+A2', 'P3-A1+A2', 'P4-A1+A2', 'Pz-A1+A2', 'T3-A1+A2', 'T4-A1+A2', 'T5-A1+A2', 'T6-A1+A2', 'O1-A1+A2', 'O2-A1+A2']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 845 samples (3.301 s)

Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) me

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Removed notch frequencies (Hz):
     50.00 : 1292 windows
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 845 samples (3.301 s)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Executing RANSAC
This may take a while, so be patient...


  0%|          |  : 0/69 [00:00<?,       ?it/s]

### 2. Downsampling

In [ ]:
raw.resample(256, npad="auto")    # set sampling frequency to 256 points per second

>according to the Nyquist frequency, sampling rate should be at least two times of frequency.
Moreover, 128 points are suitable and 256 points are desirable


### 3. Filtering the data

In [ ]:
# Apply band-pass filter from 0.5 to 80 Hz
raw.filter(l_freq=0.5, h_freq=80.0, fir_design='firwin')

# Apply notch filter at 50 Hz
raw.notch_filter(freqs=50, fir_design='firwin')


>Lw-pass filter with 1 Hz cutoff frequency for removing low-frequency drifts.
    High-pass filter with 30 Hz cutoff frequency for deteriorating the effect of
    the AC power line frequency, cell phones, the geomagnetic field and so forth.
    Therefore, a band-pass filter was used in the range 1Hz-30Hz with one step.
    You can apply another band pass filter due to your own assumtions and hypotheses.
    I recommend that band pass filtering would be better to occurr before the EEG
    data epoching and artifact removal with ICA.

### 5. Visual Inspection

###### 5.1 Plot continuous data

In [ ]:
raw.plot()

###### Remove bad channels: Have some problems

In [ ]:
# import mne
# from autoreject import AutoReject

# # Create a standard montage
# montage = mne.channels.make_standard_montage('standard_1020')

# # Rename channels in raw to match montage names if needed
# # This example assumes you have A1 and A2 suffixes; customize as needed
# raw.rename_channels(lambda x: x.replace('-A1', '').replace('-A2', ''))

# # Set the montage to the raw data
# raw.set_montage(montage, on_missing='ignore')

# # Ensure channel positions are valid after setting the montage
# if raw.info['dig'] is None or len(raw.info['dig']) == 0:
#     raise RuntimeError("No valid channel positions found. Check montage setting.")

# # Initialize AutoReject
# ar = AutoReject(n_interpolate=[1, 4, 8], consensus=[0.6, 0.7, 0.8], picks='eeg', verbose='tqdm')

# # Create fixed-length epochs
# epochs = mne.make_fixed_length_epochs(raw, duration=2.0, preload=True)

# # Apply AutoReject to find bad channels and reject or interpolate them
# epochs_clean, reject_log = ar.fit_transform(epochs, return_log=True)

# # Get bad channels detected by AutoReject
# bad_channels = reject_log.bad_channels

# # Limit to 3 bad channels (if more than 3 are detected)
# if len(bad_channels) > 3:
#     bad_channels = bad_channels[:3]

# # Drop the bad channels from the raw data
# raw.drop_channels(bad_channels)

# print(f"Bad channels detected and removed: {bad_channels}")


###### Mine 1: Peak-to-Peak Amplitude Rejection

In [ ]:
# Create fixed-length epochs
epochs = mne.make_fixed_length_epochs(raw, duration=2.0, preload=True)

# Define rejection thresholds (values in microvolts)
reject_criteria = dict(eeg=100e-6)

# Drop bad epochs based on the rejection criteria
epochs.drop_bad(reject=reject_criteria)

epochs.plot()


###### Mine 2: Frequency-Based Rejection (Power Spectrum)

In [ ]:
# # Compute the Power Spectral Density (PSD) directly from the raw object
# psds = raw.compute_psd(fmin=0.5, fmax=45)
# freqs = psds.freqs

# # Optional: Plot the PSD
# psds.plot()


###### mine 4: Signal Discontinuities and Abrupt Jumps

In [ ]:
import mne
import numpy as np

# raw = mne.io.read_raw_edf(edf_file, preload=True)

# Threshold for detecting abrupt jumps (customize as needed)
threshold = 100e-6

# Find discontinuities by calculating the difference between consecutive samples
signal_diff = np.abs(np.diff(raw.get_data()))

# Create a boolean mask for abrupt jumps exceeding the threshold
abrupt_jumps = np.any(signal_diff > threshold, axis=0)

# Get the indices of the abrupt changes
jump_indices = np.where(abrupt_jumps)[0]

# Create lists to store the onset and duration of annotations
onsets = []
durations = []

# Create annotations for these segments
for idx in jump_indices:
    start = max(0, idx - 1)  # Adjust for segment length (1 second in this example)
    end = min(raw.n_times, idx + 1)
    onsets.append(start / raw.info['sfreq'])  # onset in seconds
    durations.append((end - start) / raw.info['sfreq'])  # duration in seconds

# Convert to numpy arrays
onsets_array = np.array(onsets)
durations_array = np.array(durations)

# Create Annotations object using the correct constructor
annotations = mne.Annotations(onsets_array, durations_array, ['Jump'] * len(onsets_array))

# Add the annotations to the raw object
raw.set_annotations(annotations)

# Plot the raw data with annotations
raw.plot()

# Review the annotations
print(raw.annotations)


In [ ]:
# Create fixed-length epochs (e.g., 2 seconds) without rejection initially
epochs = mne.make_fixed_length_epochs(raw, duration=2.0, preload=True)

# Define the rejection criteria
reject_criteria = dict(eeg=100e-6)  # Adjust threshold as needed

# Apply the rejection criteria to drop bad epochs
epochs.drop_bad(reject=reject_criteria)


##### CAR

In [ ]:
# # Apply Common Average Referencing (CAR)
# raw.set_eeg_reference('average', projection=True)

# # Create fixed-length epochs after applying CAR
# epochs = mne.make_fixed_length_epochs(raw, duration=2.0, preload=True)


##### Mine: ICA

In [ ]:
# Apply ICA for artifact removal (e.g., eye blinks, muscle artifacts)
ica = mne.preprocessing.ICA(n_components=19, random_state=97, max_iter=800)
ica.fit(raw)

# Automatically detect EOG artifacts (e.g., eye blinks)
# eog_inds, eog_scores = ica.find_bads_eog(raw)
# ica.exclude = eog_inds  # Mark these components for exclusion

# Apply ICA to remove artifacts
raw_cleaned = ica.apply(raw)


In [ ]:
from scipy.io import savemat

# Get the data and the channel names
data, times = raw_cleaned.get_data(return_times=True)  # EEG data and the corresponding times
channel_names = raw_cleaned.ch_names  # Channel names
sfreq = raw_cleaned.info['sfreq']  # Sampling frequency

# Create a dictionary to save
raw_dict = {
    'data': data,  # The EEG data
    'times': times,  # Time points for the data
    'channel_names': channel_names,  # Channel names
    'sfreq': sfreq  # Sampling frequency
}

# Save the dictionary as a .mat file
# savemat('IDpz102_W2OC-Cleaned_CAR.mat', raw_dict)
savemat('IDpz102_W2OA-Cleaned.mat', raw_dict)

print("Raw data saved as 'eeg_data.mat'")


In [ ]:
import mne

# Define the duration of the epochs in seconds
epoch_duration = 2.0  # 2 seconds

# Create fixed-length epochs
epochs = mne.make_fixed_length_epochs(raw_cleaned, duration=epoch_duration, preload=True)

# Optional: Check the number of epochs created
print(f"Number of epochs created: {len(epochs)}")

# Save the epochs to disk
# epochs.save('/content/IDpz109_W1OA-epochs-2s.fif', overwrite=True)  # Adjust the file path as needed


In [ ]:
# # Reload the original EDF file (for ECG channel)
# raw_ecg = mne.io.read_raw_edf(edf_file, preload=True)

# # Pick the ECG channel only
# raw_ecg.pick_channels(['ECG1+'])

# # Apply the same filtering and reference as the EEG data
# raw_ecg.filter(0.5, 45, fir_design='firwin')  # Apply same band-pass filter
# raw_ecg.set_eeg_reference('average', projection=True).apply_proj()  # Apply same referencing

# # Now append the ECG channel back to the cleaned raw data
# raw_cleaned.add_channels([raw_ecg])

# # Continue with your workflow


In [ ]:
import numpy as np
from scipy.io import savemat

# Convert epochs to a NumPy array
data_array = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)

# Prepare a dictionary to save as .mat
mat_dict = {
    'data': data_array,
    'times': epochs.times,
    'events': epochs.events,
    'event_ids': epochs.event_id
}

# Save the dictionary as a .mat file
savemat('/content/IDpz109_W2OA-cleaned.mat', mat_dict)
